# Web Usage

Data log web yang digunakan pada code ini adalah data NASA.

In [ ]:
import pandas as pd
from datetime import timedelta

# --- Ganti 'path/ke/file_log_anda.csv' dengan lokasi file Anda ---
log_file_path = 'NASA_Jul95_cleaned.csv'
# -----------------------------------------------------------------

# Tentukan batas waktu sesi
SESSION_TIMEOUT = timedelta(minutes=30)

# Daftar ekstensi file aset yang akan difilter (dibuang)
ASSET_EXTENSIONS = [
    '.css', '.js', '.jpg', '.jpeg', '.png', '.gif', '.ico',
    '.txt', '.svg', '.woff', '.woff2', '.ttf', '.eot', '.zip'
]

def main():
    ## 1. TAHAP PREPROCESSING (Pembersihan dan Filter Data)

    print(f"Membaca log dari {log_file_path}...")

    col_names = [
        'IP', 'Logname', 'User', 'Time', 'Method',
        'URI', 'Protocol', 'Status', 'Size'
    ]

    df = None # Initialize df to None

    try:
        df = pd.read_csv(
            log_file_path,
            sep=',', header=None, names=col_names,
            skiprows=1, quotechar='"', on_bad_lines='skip', # Keep on_bad_lines
            engine='python' # Added engine='python' for more robust parsing
        )
    except FileNotFoundError:
        print(f"Error: File tidak ditemukan di {log_file_path}")
        return # Use return instead of exit() for Colab compatibility
    except Exception as e:
        print(f"Error saat membaca file: {e}")
        return # Use return instead of exit() for Colab compatibility

    # Check if df was successfully loaded before proceeding
    if df is None:
        print("Dataframe could not be loaded. Stopping execution.")
        return

    print("Pratinjau data mentah berhasil dibaca.")
    print("-" * 50)

    # --- TAMBAHAN: INFORMASI STATISTIK DATA MENTAH ---
    print(f"📊 Jumlah baris data mentah (Awal): {len(df)}")

    # 1. Cek Top User (IP paling sering muncul)
    ip_counts = df['IP'].value_counts()
    print("\n🏆 Top 10 IP dengan akses terbanyak:")
    print(ip_counts.head(10))

    # 2. Cek Jumlah IP Duplikat (Muncul > 1 kali)
    dupe_ips = ip_counts[ip_counts > 1]
    print(f"\n⚠️ Jumlah Unik IP: {len(ip_counts)}")
    print(f"⚠️ Jumlah IP yang 'Duplikat' (akses > 1 kali): {len(dupe_ips)}")
    print("-" * 50)

    # --- TAMBAHAN: INFORMASI NILAI NAN ---
    print("\n🔍 Mengecek nilai NaN dalam dataset mentah...")
    nan_counts = df.isnull().sum()
    nan_percentages = (df.isnull().sum() / len(df)) * 100
    nan_info = pd.DataFrame({'NaN Count': nan_counts, 'NaN Percentage': nan_percentages})
    print(nan_info[nan_info['NaN Count'] > 0].to_string()) # Hanya tampilkan kolom dengan NaN
    print("Total NaN values in raw data:", df.isnull().sum().sum())
    print("-" * 50)


    print("Memulai preprocessing (filter GET, 200, dan aset)...")

    # 1. Filter: Hanya ambil Request Method 'GET'
    df_filtered = df[df['Method'] == 'GET'].copy()

    # 2. Filter: Hanya ambil Status '200' (OK)
    df_filtered['Status'] = pd.to_numeric(df_filtered['Status'], errors='coerce')
    df_filtered = df_filtered.dropna(subset=['Status'])
    df_filtered = df_filtered[df_filtered['Status'] == 200].copy()

    # 3. Filter: "filter html" (buang file aset)
    lower_uri = df_filtered['URI'].astype(str).str.lower()
    # is_asset = lower_uri.apply(lambda x: any(x.endswith(ext) for ext in ASSET_EXTENSIONS))
    # # 'df_clean' sekarang berisi data bersih DENGAN SEMUA KOLOM ASLI
    # df_clean = df_filtered[~is_asset].copy()

    # Langsung ambil baris yang URI-nya berakhiran '.html'
    # Perbaikan: Gunakan df_filtered dan pastikan tidak ada NaN di kolom URI sebelum filter
    df_clean = df_filtered[df_filtered['URI'].fillna('').str.lower().str.endswith('.html')].copy()

    # Konversi kolom 'Time' ke format datetime
    print("Mengonversi waktu (dengan format ISO8601)...")
    df_clean['Time'] = pd.to_datetime(df_clean['Time'], format='ISO8601')

    print("Data setelah preprocessing (sebelum pengelompokan sesi):")
    print(df_clean.head().to_string())
    print("-" * 50)

    # --- TAMBAHAN: INFORMASI SETELAH FILTERING ---
    jumlah_awal = len(df)
    jumlah_akhir = len(df_clean)
    jumlah_dibuang = jumlah_awal - jumlah_akhir

    print(f"✅ Jumlah data setelah diproses (df_clean): {jumlah_akhir}")
    print(f"🗑️ Jumlah data 'sampah' (aset/error) yang dibuang: {jumlah_dibuang}")
    print(f"📉 Persentase penyusutan data: {(jumlah_dibuang/jumlah_awal)*100:.2f}%")
    print("-" * 50)


    ## 2. TAHAP IDENTIFIKASI SESI (Sessionization)

    print(f"Mengelompokkan sesi (timeout: {SESSION_TIMEOUT})...")

    # Urutkan data (PENTING)
    df_clean = df_clean.sort_values(by=['IP', 'Time'])

    # Hitung selisih waktu
    time_diff = df_clean.groupby('IP')['Time'].diff()
    is_new_session = (time_diff.isna()) | (time_diff > SESSION_TIMEOUT)

    # Buat kolom SessionID langsung di 'df_clean'
    df_clean['SessionID'] = is_new_session.cumsum()

    # --- OUTPUT BARU ---
    print("\n--- PRATINJAU LOG DENGAN SESI ---")
    # Menampilkan kolom-kolom yang relevan
    cols_to_show = ['SessionID', 'IP', 'Time', 'URI', 'Status']
    print(df_clean[cols_to_show].head(15).to_string())
    print("-" * 50)

    # Menyimpan file log yang sudah dikelompokkan
    try:
        df_clean.to_csv('hasil_log_dengan_sesiNASA.csv', index=False)
        print("\nData log yang sudah dikelompokkan disimpan ke 'hasil_log_dengan_sesiNASA.csv'")
    except Exception as e:
        print(f"\nGagal menyimpan file 'hasil_log_dengan_sesiNASA.csv': {e}")
    # ----------------------------------------------


    ## 3. TAHAP PENYUSUNAN MATRIKS (Data Structuring)

    print("\nMembuat matriks Sesi x Halaman...")

    # Kita menggunakan 'df_clean' yang sekarang sudah punya SessionID
    matrix = pd.crosstab(
        index=[df_clean['SessionID'], df_clean['IP']],
        columns=df_clean['URI']
    )
    matrix_binary = (matrix > 0).astype(int)
    final_output = matrix_binary.reset_index().drop('SessionID', axis=1)

    print("\n--- HASIL AKHIR (MATRIKS) ---")
    print(final_output.to_string())

    # Menyimpan file matriks
    try:
        final_output.to_csv('hasil_matriks_sesiNASA.csv', index=False)
        print("\nMatriks hasil juga disimpan ke 'hasil_matriks_sesiNASA.csv'")
    except Exception as e:
        print(f"\nGagal menyimpan file 'hasil_matriks_sesiNASA.csv': {e}")

main()

Membaca log dari NASA_Jul95_cleaned.csv...
Pratinjau data mentah berhasil dibaca.
--------------------------------------------------
📊 Jumlah baris data mentah (Awal): 1891714

🏆 Top 10 IP dengan akses terbanyak:
IP
piweba3y.prodigy.com    17572
piweba4y.prodigy.com    11591
piweba1y.prodigy.com     9868
alyssa.prodigy.com       7852
siltb10.orl.mmc.com      7573
piweba2y.prodigy.com     5922
edams.ksc.nasa.gov       5434
163.206.89.4             4906
news.ti.com              4863
disarray.demon.co.uk     4353
Name: count, dtype: int64

⚠️ Jumlah Unik IP: 81982
⚠️ Jumlah IP yang 'Duplikat' (akses > 1 kali): 76282
--------------------------------------------------

🔍 Mengecek nilai NaN dalam dataset mentah...
Empty DataFrame
Columns: [NaN Count, NaN Percentage]
Index: []
Total NaN values in raw data: 0
--------------------------------------------------
Memulai preprocessing (filter GET, 200, dan aset)...
Mengonversi waktu (dengan format ISO8601)...
Data setelah preprocessing (sebelum pe